In [30]:
import pandas as pd
from datasets import load_dataset, DatasetDict, Dataset

sat_math_dict = load_dataset("knazarali3/group2_processed_sat_math_cot")
train_dataset = sat_math_dict["train"]
test_dataset = sat_math_dict["test"]
test_dataset_df = pd.DataFrame(test_dataset)
test_dataset_df["row_index"]  = test_dataset_df.index

In [2]:
all_models_questions_got_wrong_sat_math = pd.read_csv("all_models_questions_got_wrong_sat_math.csv")
all_models_questions_got_wrong_sat_math

,Unnamed: 0,Question Id,llama_is_correct,Llama Model Solution Numbers,llema_is_correct,llema_solution,qwen_is_correct,Qwen Model Solution Numbers,mistral_is_correct,Mistral Model Solution Numbers,all_incorrect
0,0,27493,False,[711],False,NaN,False,[561],False,[5958],True
1,3,22729,False,[11],False,NaN,False,[12],False,[801],True
2,5,9552,False,"[105, 15]",False,NaN,False,[1575],False,[],True
3,10,19577,False,[9444],False,25.0,False,"[95, 95]",False,[95],True
4,12,26373,False,"[30, 60]",False,4.0,False,[3],False,[3],True
...,...,...,...,...,...,...,...,...,...,...,...
1420,6148,3246,False,[4],False,11.0,False,[3],False,[6],True
1421,6150,24214,False,"[2, 1, 31]",False,NaN,False,[31],False,[4],True
1422,6173,24952,False,[32],False,NaN,False,[51],False,[41],True
1423,6187,16329,False,"[3, 7]",False,NaN,False,[1],False,[1],True


In [31]:
test_dataset_df[test_dataset_df["id"] == "27493"]

## llema
with open("inference_sat_math_results/llemma_math_sat_answers.txt", "r") as file:
    data = [eval(line.strip()) for line in file]
llema = pd.DataFrame(data)

## llama
llama = pd.read_csv("inference_sat_math_results/the_inference_llama3_sat_math.csv")

## qwen
qwen = pd.read_csv("inference_sat_math_results/inference_QWEN_sat_math.csv")
qwen = qwen.merge(test_dataset_df, left_on='row_index', right_on='row_index')

## mistral
mistral = pd.read_csv("inference_sat_math_results/Mistral_Inference_SAT_Dataset.csv")


In [16]:
test_dataset_df[test_dataset_df["id"]=="27493"]["answer"][0]

'There were 50 people there.'

In [20]:
list(llema[llema["Question Id"] == "27493"]["Generated Answer"])

['Let $n$ be the number of people at the movie. Then, the cost of food is $3n$, the cost of equipment is $2(1200+3n)$, and the total cost is $1200+3n+2(1200+3n)=5400+7n$. The cost of the movie is $10,000$, and the profit is $5950$, so the total revenue is $10,000+5950=16,950$. Therefore, the total cost is']

In [18]:
llama[llama["ID"] == "27493"]["Run_1"][0]

'There were 711 people who got food.'

In [35]:
qwen[qwen["id"] == "27493"]["Run_1"][0]

'561'

In [ ]:
output_dir = "mrush30/cs6220-llemma_both"
peft_model = AutoPeftModelForCausalLM.from_pretrained(
  output_dir,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(output_dir)
pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer)